#1
# Prediction of Traffic Accident Risk
Team 7: 어서오십쇼HUMAN  
Editor: 유성민, 김도운

#2
## 1. 데이터 전처리

In [1]:
#3
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
#3


# preprocessing = pd.read_csv('../input/finalres.csv')

# preprocessing = preprocessing.iloc[np.random.permutation(len(preprocessing))]

# y = preprocessing.loc[:, preprocessing.columns == '사고내용']
# y = y.astype('float')
# x = preprocessing.loc[:, preprocessing.columns != '사고내용']

# x = x[:30000]
# y = y[:30000]
newx = pd.read_csv('../input/newx.csv')
newy = pd.read_csv('../input/newy.csv')
newy = newy.astype('float')
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(newx, newy, test_size=0.2,random_state = 1)

In [9]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
knn.score(x_test,y_test)

0.8027587735091266

In [10]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

0.8703431847768883

#5
# Do you need standardization?

In [20]:
%%javascript
//6
var input = prompt('Do you need min max scale? then if you want input y', 'waiting your input ......');
var i;
if(input == 'y'){
     alert('start with min max scaler');
    Jupyter.notebook.execute_cells([3]); // read data
     for(i=42; i<=68; i++) {
        Jupyter.notebook.execute_cells([i]); // learning
     }
}else{
    alert('start without min max scaler.');
      Jupyter.notebook.execute_cells([3]); // read data
     for(i=7; i<=34; i++) {
        Jupyter.notebook.execute_cells([i]); // learning
     }
}

<IPython.core.display.Javascript object>

#7
## 2. 모델 학습

#8
# Prepare Model training and analysis

> editor: seongminyoo   
> Date of modification: 2020-05-25 04:49pm

`solving problem`
+ Classification Problem

`using model`
+ KNN
+ Decision Tree
+ Bagging
+ Random Forest
+ SVM
+ Neural Network classifier

`algorithm for each step`
1. GridSearchCV by 10 fold or 5 fold
    - make parameter set
2. Test
    - best_esitimater is model that construct best hyper parameter set
3. Analysis result
    - use confusion matrix

#9
# Make functions

In [11]:
#10
# 최고 모델 만드는 함수

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import balanced_accuracy_score

def make_model(model, param_grid, cv):
    try: # random_state 가능 모델
        gs = GridSearchCV(estimator=model(random_state = 20000), param_grid=param_grid, 
                          scoring='accuracy', cv=cv)
    except Exception: # random_state 불가능 모델
        print('error detection!')
        gs = GridSearchCV(estimator=model(), param_grid=param_grid, 
                          scoring='accuracy', cv=cv)
    finally:#베스트 파라미터 출력, 최적의 하이퍼 파라미터로 구성된 모델 리턴
        gs.fit(x_train,y_train)
        best_params = gs.best_params_
        print(best_params)
        return gs

# test function
def make_test(model): # confusion matrix와 성능을 측정하고 출력
        best_model=model.best_estimator_
        y_pred = best_model.predict(x_test)
        print(classification_report(y_test,y_pred,target_names=['class 0','class 1','class 2']))
        cm = confusion_matrix(y_test,y_pred)
        print('Confusion matrix: \n',cm)

# 50개의 Thread를 생성하여 병렬처리 -시간 매우 단축-

In [12]:
#11
from concurrent.futures import as_completed, ProcessPoolExecutor
import time
import numpy as np
import winprocess
#프로세스 50개를 만들어서 시간을 거의 10배 단축
def parallel_processing(model, param_grid, cv):
    executor = ProcessPoolExecutor(max_workers=50)
    fs = winprocess.submit(executor, make_model,model, param_grid, cv)
    return fs.result()

#12
# KNN

In [5]:
#13
# hyper parameter sets
param_range = [3,5,7]
param_pval = [1,2]
param_grid = [
    {'n_neighbors': param_range, 'p': param_pval}
]

In [ ]:
import time
start = time.time()
make_model(KNeighborsClassifier,param_grid, 10)
print('걸린 시간은 '+str(time.time()-start)+'초')

In [ ]:
#14
start = 0
start = time.time()
best_model = parallel_processing(KNeighborsClassifier,param_grid, 10)
print('걸린 시간은 '+str(time.time()-start)+'초')

In [ ]:
#15
make_test(best_model)

#16
# Decision Tree

In [ ]:
#17
# hyper parameter sets
param_range = [5,8,11,13]
param_grid = [
    {'max_depth': param_range, 'criterion': ['entropy']},
    {'max_depth': param_range, 'criterion': ['gini']}]

In [ ]:
#18
# Decision Tree train and test
best_model = parallel_processing(tree.DecisionTreeClassifier,param_grid,10)

In [ ]:
#19
make_test(best_model)

#20
# Random Forest

In [13]:
#21
# hyper parameter set for Random Forest
estimators = [100]
depth_range = [5,8,11,13,26]
param_grid = [
    {'max_depth': depth_range, 'criterion': ['entropy'], 'n_estimators':estimators},
    {'max_depth': depth_range, 'criterion': ['gini'], 'n_estimators':estimators}]

In [14]:
#22
# Random Forest train and test
best_model = parallel_processing(RandomForestClassifier,param_grid,10)

In [15]:
#23
make_test(best_model)

              precision    recall  f1-score   support

     class 0       0.97      0.99      0.98     12079
     class 1       0.94      0.94      0.94     12015
     class 2       0.95      0.92      0.94     12009

    accuracy                           0.95     36103
   macro avg       0.95      0.95      0.95     36103
weighted avg       0.95      0.95      0.95     36103

Confusion matrix: 
 [[11958    68    53]
 [  219 11248   548]
 [  213   703 11093]]


#24
# Bagging

In [14]:
#25
# hyper parameter set for Bagging
estimators = [10, 20]
depth_range = [3,5,8,11,13]
param_grid = [
    {'base_estimator__max_depth': depth_range, 'base_estimator__criterion': ['entropy'], 'n_estimators':estimators},
    {'base_estimator__max_depth': depth_range, 'base_estimator__criterion': ['gini'], 'n_estimators':estimators}
 ]

In [15]:
#26
#Bagging train and test

gs = GridSearchCV(estimator=BaggingClassifier(tree.DecisionTreeClassifier(random_state = 1),random_state=20000)
                  , param_grid=param_grid, scoring='accuracy', cv=10)
gs.fit(x_train,y_train)
print(gs.best_params_)
best_bg=gs.best_estimator_
y_pred = best_bg.predict(x_test) 
print(classification_report(y_test,y_pred,target_names=['class 1','class 2','class 3']))
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)

KeyboardInterrupt: 

#27
# SVM

In [ ]:
#28
# hyper parameter set for SVM
param_range = [0.01, 1]
param_grid = [
    {'C': param_range, 'kernel': ['linear']},
    {'C': param_range, 'kernel': ['rbf']}]

In [ ]:
#29
# SVM Forest train and test
best_model = parallel_processing(SVC,param_grid,5)

In [ ]:
#30
make_test(best_model)

#31
# Neural Network Classifier

In [16]:
#32
# hyper parameter set for Neural Network
param_learning_rate=['constant', 'adaptive']
param_solver=['sgd', 'adam']
param_hidden_layer_sizes=[(100,)]
param_grid = [
    {'learning_rate': param_learning_rate, 'hidden_layer_sizes':param_hidden_layer_sizes,'solver':param_solver}]

In [ ]:
#33
# Neural Network train and test
best_model = parallel_processing(MLPClassifier,param_grid, 10)

In [ ]:
#34
make_test(best_model)

In [3]:
model = MLPClassifier()
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print(classification_report(y_test,y_pred,target_names=['class 1','class 2','class 3']))
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)

              precision    recall  f1-score   support

     class 1       0.70      0.88      0.78     12079
     class 2       0.58      0.47      0.52     12015
     class 3       0.63      0.58      0.60     12009

    accuracy                           0.65     36103
   macro avg       0.64      0.65      0.64     36103
weighted avg       0.64      0.65      0.64     36103

Confusion matrix: 
 [[10672   927   480]
 [ 2652  5695  3668]
 [ 1870  3165  6974]]


# class imbalance에 적합한 모델 선정 중 -테스트-

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
logreg = LogisticRegression()

logreg.fit(x_train,y_train)

y_pred  = logreg.predict(x_test)

print(classification_report(y_test,y_pred,target_names=['class 0','class 1','class 2']))
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)

In [ ]:
sgd = SGDClassifier()
sgd.fit(x_train, y_train)
y_pred = sgd.predict(x_test)
print(classification_report(y_test,y_pred,target_names=['class 0','class 1','class 2']))
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)

In [31]:
gaussian = GaussianNB()
gaussian.fit(y_train, y_train)
y_pred = gaussian.predict(x_test)
print(classification_report(y_test,y_pred,target_names=['class 0','class 1','class 2']))
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00       528
     class 1       0.17      0.07      0.10      5026
     class 2       0.85      0.95      0.90     31168

    accuracy                           0.81     36722
   macro avg       0.34      0.34      0.33     36722
weighted avg       0.75      0.81      0.77     36722

Confusion matrix: 
 [[    0    28   500]
 [    5   353  4668]
 [   36  1668 29464]]


In [28]:
perceptron = Perceptron()
perceptron.fit(x_train, y_train)
y_pred = perceptron.predict(x_test)
print(classification_report(y_test,y_pred,target_names=['class 0','class 1','class 2']))
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00        29
     class 1       0.00      0.00      0.00       292
     class 2       0.84      1.00      0.91      1679

    accuracy                           0.84      2000
   macro avg       0.28      0.33      0.30      2000
weighted avg       0.70      0.84      0.77      2000

Confusion matrix: 
 [[   0    0   29]
 [   0    0  292]
 [   0    0 1679]]


In [29]:
linear_svc = LinearSVC()
linear_svc.fit(x_train, y_train)
y_pred = linear_svc.predict(x_test)
print(classification_report(y_test,y_pred,target_names=['class 0','class 1','class 2']))
cm = confusion_matrix(y_test,y_pred)
print('Confusion matrix: \n',cm)

              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00        29
     class 1       0.00      0.00      0.00       292
     class 2       0.84      1.00      0.91      1679

    accuracy                           0.84      2000
   macro avg       0.28      0.33      0.30      2000
weighted avg       0.70      0.84      0.77      2000

Confusion matrix: 
 [[   0    0   29]
 [   0    0  292]
 [   0    0 1679]]


# standard scaler pipeline적용된 모델 학습

#35
# Prepare Model training and analysis

> editor: seongminyoo   
> Date of modification: 2020-05-25 04:49pm

`solving problem`
+ Classification Problem

`using model`
+ KNN
+ Decision Tree
+ Bagging
+ Random Forest
+ SVM
+ Neural Network classifier

`algorithm for each step`
1. Make pipeline
    - standard data, model
2. GridSearchCV by 10 fold or 5 fold
    - make parameter set
3. Test
    - best_esitimater is model that construct best hyper parameter set
4. Analysis result
    - use confusion matrix

#36
# Make functions

In [30]:
#37
def make_model(model_name, model, param_grid, cv):
    try: # random_state 가능 모델
        pipe_svc = Pipeline([('mms', MinMaxScaler()), (model_name,model(random_state = 1))])
    except Exception: # random_state 불가능 모델
        print('error detection!')
        pipe_svc = Pipeline([('mms', MinMaxScaler()), (model_name,model())])
    finally:
        gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=cv)
        gs.fit(x_train,y_train)
        best_params = gs.best_params_
        print(best_params)
        return gs

# test function
def make_test(model):
        best_model=model.best_estimator_
        y_pred = best_model.predict(x_test)
        print(classification_report(y_test,y_pred,target_names=['class 1','class 2','class 3']))
        cm = confusion_matrix(y_test,y_pred)
        print('Confusion matrix: \n',cm)

In [31]:
#38
from concurrent.futures import as_completed, ProcessPoolExecutor
import time
import numpy as np
import winprocess

def parallel_processing(model_name, model, param_grid, cv):
    executor = ProcessPoolExecutor(max_workers=50)
    fs = winprocess.submit(executor, make_model, model_name, model, param_grid, cv)
    return fs.result()

#39
# KNN

In [32]:
#40
# hyper parameter sets
param_range = [i for i in range(3,10)]
param_grid = [
    {'knn__n_neighbors': param_range, 'knn__metric': ['euclidean']},
    {'knn__n_neighbors': param_range, 'knn__metric': ['manhattan']}]

In [33]:
#41
best_model = parallel_processing('knn',KNeighborsClassifier,param_grid, 5)

In [34]:
#42
make_test(best_model)

              precision    recall  f1-score   support

     class 1       0.87      0.97      0.92      2033
     class 2       0.70      0.74      0.72      1978
     class 3       0.76      0.63      0.69      1989

    accuracy                           0.78      6000
   macro avg       0.78      0.78      0.78      6000
weighted avg       0.78      0.78      0.78      6000

Confusion matrix: 
 [[1976   30   27]
 [ 153 1464  361]
 [ 132  611 1246]]


#43
# Decision Tree

In [26]:
#44
# hyper parameter sets
depth_range = [27,54]
param_grid = [
    {'dt__max_depth': param_range, 'dt__criterion': ['entropy']},
    {'dt__max_depth': param_range, 'dt__criterion': ['gini']}]

In [27]:
#45
# Decision Tree train and test
best_model = parallel_processing('dt',tree.DecisionTreeClassifier,param_grid,5)

In [28]:
#46
make_test(best_model)

              precision    recall  f1-score   support

     class 1       0.64      0.77      0.70       662
     class 2       0.59      0.32      0.41       650
     class 3       0.59      0.73      0.65       688

    accuracy                           0.61      2000
   macro avg       0.60      0.60      0.59      2000
weighted avg       0.60      0.61      0.59      2000



#47
# Random Forest

In [ ]:
#48
# hyper parameter set for Random Forest
estimators = [10,100]
depth_range = [i for i in range(3,7)]
param_grid = [
    {'rf__max_depth': depth_range, 'rf__criterion': ['entropy'], 'rf__n_estimators':estimators},
    {'rf__max_depth': depth_range, 'rf__criterion': ['gini'], 'rf__n_estimators':estimators}]

In [ ]:
#49
# Random Forest train and test
best_model = parallel_processing('rf',RandomForestClassifier,param_grid,5)

In [ ]:
#50
make_test(best_model)`

#51
# Bagging

In [ ]:
#52
# hyper parameter set for Bagging
estimators = [10, 100]
depth_range = [i for i in range(3,7)]
param_grid = [
    {'bg__base_estimator__criterion': ['entropy'], 'bg__n_estimators':estimators},
    {'bg__base_estimator__criterion': ['gini'], 'bg__n_estimators':estimators}
 ]

In [ ]:
#53
#Bagging train and test
pipe_svc = Pipeline([('scl', StandardScaler()), ('bg',BaggingClassifier(tree.DecisionTreeClassifier(random_state = 1),random_state=1))])
gs = GridSearchCV(estimator=pipe_svc, param_grid=param_grid, scoring='accuracy', cv=5,)
gs.fit(x_train,y_train)
print(gs.best_params_)
best_bg=gs.best_estimator_
y_pred = best_bg.predict(x_test) 
print(classification_report(y_test,y_pred,target_names=['lsclass 1','class 2','class 3']))

#54
# SVM

In [ ]:
#55
# hyper parameter set for SVM
param_range = [0.01, 1 ,10.0]
param_grid = [
    {'clf__C': param_range, 'clf__kernel': ['linear']},
    {'clf__C': param_range, 'clf__kernel': ['rbf']}]

In [ ]:
#56
# SVM Forest train and test
best_model = parallel_processing('clf',SVC,param_grid,5)

In [ ]:
#57
make_test(best_model)

#58
# Neural Network Classifier

In [ ]:
#59
# hyper parameter set for Neural Network
param_learning_rate=['constant', 'adaptive']
param_activation = ['relu', 'identity', 'tanh', 'logistic']
param_solver=['sgd', 'adam']
param_hidden_layer_sizes=[(100,)]
param_grid = [
    {'nnc__learning_rate': param_learning_rate, 'nnc__hidden_layer_sizes':param_hidden_layer_sizes, 
     'nnc__activation':param_activation, 'nnc__solver':param_solver}
]

In [ ]:
#60
# Neural Network train and test
best_model = parallel_processing('nnc',MLPClassifier,param_grid, 5)

In [ ]:
#61
make_test(best_model)